In [1]:
import torch
import torch.nn as nn
import scipy.io
import torch.nn.functional as F
from torch.autograd import Variable

In [2]:
def to_var(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)

In [3]:
dd=torch.randn(10,10,128) 


In [ ]:
class CNN_1(nn.Module):
    def __init__(self, hidden_size, dropout_rate=0.5):
        super(CNN_1, self).__init__()
        self.hidden_size = hidden_size
        self.dropout = nn.Dropout(dropout_rate)
        self.relu = nn.ReLU()

        self.conv1_1 = nn.Conv1d(1, 8, kernel_size=8, stride=2, padding=3)#输入3840，输出1920
        self.conv1_2 = nn.Conv1d(1, 8, kernel_size=16, stride=2, padding=7)#输入3840，输出1920
        self.conv1_3 = nn.Conv1d(1, 8, kernel_size=32, stride=2, padding=15)#输入3840，输出1920
        self.conv1_4 = nn.Conv1d(1, 8, kernel_size=64, stride=2, padding=31)#输入3840，输出1920

        self.norm1 = nn.BatchNorm1d(32) # 1920   所以4*8=32个通道
        self.pool1 = nn.MaxPool1d(6, stride=4, padding=1, return_indices=True) # 480  所以1920/4=480

        self.conv2_1 = nn.Conv1d(32, 16, kernel_size=3, stride=1, padding=1) #尺寸不变
        self.conv2_2 = nn.Conv1d(32, 16, kernel_size=5, stride=1, padding=2) #尺寸不变

        self.norm2 = nn.BatchNorm1d(32)   #所以又16*2=32通道
        self.pool2 = nn.MaxPool1d(3, stride=2, padding=1, return_indices=True) # 240


        self.conv3_1 = nn.Conv1d(32, 16, kernel_size=3, stride=1, padding=1)
        self.conv3_2 = nn.Conv1d(32, 16, kernel_size=5, stride=1, padding=2)

        self.norm3 = nn.BatchNorm1d(32)
        self.pool3 = nn.MaxPool1d(3, stride=2, padding=1, return_indices=True) # 120 x 32  大小120，通道32

        self.fc = nn.Linear(61*32, hidden_size)

    def forward(self, x):
        # batch x 1 x raw_feature

        encoded = torch.cat([self.conv1_1(x),self.conv1_2(x),self.conv1_3(x),self.conv1_4(x)],1)
        encoded = self.relu(self.norm1(encoded))
        encoded, indices1 = self.pool1(encoded)

        encoded = torch.cat([self.conv2_1(encoded),self.conv2_2(encoded)],1)
        encoded = self.relu(self.norm2(encoded))
        encoded, indices2 = self.pool2(encoded)

        encoded = torch.cat([self.conv3_1(encoded),self.conv3_2(encoded)],1)
        encoded = self.relu(self.norm3(encoded))
        encoded, indices3 = self.pool3(encoded)

        encoded = self.relu(self.fc(encoded.view(encoded.size(0),-1))) # batch x hidden_size
        return encoded


In [ ]:
class CNN_0(nn.Module):
    def __init__(self, hidden_size, dropout_rate=0.5):
        super(CNN_0, self).__init__()
        self.hidden_size = hidden_size
        self.layer1 = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=5, padding=2),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(2))
        self.layer2 = nn.Sequential(
            nn.Conv1d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(2))
        self.dropout = nn.Dropout(dropout_rate)

        self.relu = nn.ReLU()


        self.conv1_1 = nn.Conv2d(1, 8, kernel_size=(1,8), stride=(1,2), padding=(0,3))
        self.conv1_2 = nn.Conv2d(1, 8, kernel_size=(1,16), stride=(1,2), padding=(0,7))
        self.conv1_3 = nn.Conv2d(1, 8, kernel_size=(1,32), stride=(1,2), padding=(0,15))
        self.conv1_4 = nn.Conv2d(1, 8, kernel_size=(1,64), stride=(1,2), padding=(0,31))

        self.norm1 = nn.BatchNorm2d(32) #
        self.pool1 = nn.AvgPool2d((1,6), stride=(1,4), padding=(0,1)) #

        self.conv2_1 = nn.Conv2d(32, 16, kernel_size=3, stride=1, padding=1)
        self.conv2_2 = nn.Conv2d(32, 16, kernel_size=5, stride=1, padding=2)

        self.norm2 = nn.BatchNorm2d(32)
        self.pool2 = nn.AvgPool2d((1,3), stride=(1,2), padding=(0,1)) #


        self.conv3_1 = nn.Conv2d(32, 16, kernel_size=3, stride=1, padding=1)
        self.conv3_2 = nn.Conv2d(32, 16, kernel_size=5, stride=1, padding=2)

        self.norm3 = nn.BatchNorm2d(32)
        self.pool3 = nn.AvgPool2d((5,3), stride=(5,2), padding=(0,1)) #

        self.fc = nn.Linear(61*32, hidden_size) #


    def forward(self, x):
        # shape: batch x 1 x channel5 x raw_feature1950

        encoded = torch.cat([self.conv1_1(x),self.conv1_2(x),self.conv1_3(x),self.conv1_4(x)],1)
        encoded = self.relu(self.norm1(encoded))
        encoded = self.pool1(encoded)

        encoded = torch.cat([self.conv2_1(encoded),self.conv2_2(encoded)],1)
        encoded = self.relu(self.norm2(encoded))
        encoded = self.pool2(encoded)

        encoded = torch.cat([self.conv3_1(encoded),self.conv3_2(encoded)],1)
        encoded = self.relu(self.norm3(encoded))
        encoded = self.pool3(encoded)

        encoded = self.relu(self.fc(encoded.view(encoded.size(0),-1))) # batch x hidden_size
        return encoded


In [ ]:
class Attention_v(nn.Module):
    def __init__(self, hidden_size):
        super(Attention_v, self).__init__()
        self.hidden_size = hidden_size
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()

        self.rg = nn.Linear(hidden_size,1)
        self.rc = nn.Linear(hidden_size,1)
        self.attention_layer = nn.Linear(self.hidden_size,1)

    def forward(self, h_c, h_g):
        # shape h_c: batch x channel x hidden_size
        # shape h_g: batch x hidden_size
        batch_size = h_c.size(0)
        num_channel = h_c.size(1)

        scores = to_var(torch.zeros((batch_size, num_channel))) # batch x channel

        for i in range(num_channel):
            scores[:,i] = self.energy(h_g,h_c[:,i,:])
        scores = F.softmax(scores)
        scores = scores.view(scores.size(0),scores.size(1),-1) # batch x channel x 1
        scores = scores.expand(scores.size(0),scores.size(1),self.hidden_size) # batch x channel x hidden_size

        context = h_c * scores
        context_vector = context.sum(1) # batch x hidden_size

        out = torch.cat([context_vector,h_g],1) # batch x 2*hidden_size
        return out, scores

    def energy(self, hidden_i, pre_hidden_i): # batch x hidden_size

        rate = self.sigmoid(self.rg(hidden_i) + self.rc(pre_hidden_i))

        rate = rate.view(rate.size(0),-1)
        rate = rate.expand(rate.size(0),self.hidden_size)
        out_temp = (1-rate) * hidden_i + rate * pre_hidden_i

        energies = self.attention_layer(out_temp)
        energies= energies.view(energies.size(0)) # new added due to some reasons

        return energies


In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout=0.5):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = dropout

        self.gru = nn.GRU(input_size, hidden_size, num_layers, dropout=self.dropout,
                            batch_first=True, bidirectional=True)
    def forward(self, x, hidden=None):
        # shape: batch x seq x input_size
        out, hidden = self.gru(x, hidden)
        return out


In [4]:
class Attention_t(nn.Module):
    def __init__(self, hidden_size):
        super(Attention_t, self).__init__()
        self.hidden_size = hidden_size
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()


        self.rg = nn.Linear(hidden_size,1)
        self.rc = nn.Linear(hidden_size,1)
        self.attention_layer = nn.Linear(self.hidden_size,1)

    def forward(self, h):
        # shape: batch x seq x hidden_size
        batch_size = h.size(0)
        seq_size = h.size(1)

        context_vector = to_var(torch.zeros((batch_size, seq_size, self.hidden_size)))

        for i in range(h.size(1)):
            h_i = h[:,i,:] # current hidden state: batch x hidden_size
            if i == 0:
                scores = to_var(torch.zeros((batch_size, 1)))
                pre_h_i = h[:, i, :] * 0.0 # batch x hidden_size
            else:
                scores = to_var(torch.zeros((batch_size, i)))
                pre_h_i = h[:, :i, :] # previous hidden states: batch x sub_seq_size x hidden_size
                for j in range(pre_h_i.size(1)):
                    scores[:,j] = self.energy(h_i, pre_h_i[:,j,:]) # batch x 1

            scores = F.softmax(scores) # batch x sub_seq_size

            scores = scores.view(scores.size(0),scores.size(1),-1) # batch x sub_seq_size x 1
            scores = scores.expand(scores.size(0),scores.size(1),self.hidden_size) # batch x sub_seq_size x hidden_size

            context = pre_h_i * scores # batch x sub_seq_size x hidden_size
            context_vector[:,i,:] = context.sum(1) # batch x hidden_size

        out = torch.cat([context_vector,h],2) # batch x seq x hidden_size
        return out,scores

    def energy(self, hidden_i, pre_hidden_i): # batch x hidden_size

        rate = self.sigmoid(self.rg(hidden_i) + self.rc(pre_hidden_i))

        rate = rate.view(rate.size(0),-1)
        rate = rate.expand(rate.size(0),self.hidden_size)
        out_temp = (1-rate) * hidden_i + rate * pre_hidden_i

        energies = self.attention_layer(out_temp)
        energies= energies.view(energies.size(0)) # new added due to some reasons

        return energies


In [ ]:
djgnet=Net(input_size=3840, hidden1_size=128, hidden2_size=128, num_layers=2, num_classes=5, num_channel=5)

In [5]:
yy=djgnet(dd)
print(type(yy))
print(len(yy))
print(yy.shape)

c:\users\djg\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


RuntimeError: expected type torch.FloatTensor but got torch.cuda.FloatTensor